## You can open the tutorial in a [COLAB](https://colab.research.google.com/github/deepmipt/dp_tutorials/blob/master/russian_tutorials/Tutorial_2_Seq2seq_bot.ipynb)

In [1]:
# for those who work in colab only
!pip install deeppavlov

     |████████████████████████████████| 686kB 3.4MB/s 
     |████████████████████████████████| 12.2MB 44.0MB/s 
     |████████████████████████████████| 11.8MB 32.6MB/s 
     |████████████████████████████████| 51kB 28.3MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 51kB 24.7MB/s 
     |████████████████████████████████| 12.4MB 44.1MB/s 
     |████████████████████████████████| 3.4MB 46.5MB/s 
     |████████████████████████████████| 92kB 35.3MB/s 
     |████████████████████████████████| 31.2MB 1.3MB/s 
     |████████████████████████████████| 4.1MB 50.2MB/s 
     |████████████████████████████████| 61kB 19.5MB/s 
     |████████████████████████████████| 61kB 28.9MB/s 
     |████████████████████████████████| 8.0MB 697kB/s 
     |████████████████████████████████| 307kB 51.1MB/s 
     |████████████████████████████████| 7.1MB 49.6MB/s 
     |████████████████████████████████| 143kB 55.5MB/s 
     |████████████████████████████████| 61kB 31.3MB/s 

### Туториал 2: Sequence-to-sequence Бот

In this tutorial we are going to implement [sequence-to-sequence](https://arxiv.org/abs/1409.3215) model in DeepPavlov.

Sequence-to-sequence is the concept of mapping input sequence to target sequence. Sequence-to-sequence models consist of two main components: encoder and decoder. Encoder is used to encode the input sequence to dense representation and decoder uses this dense representation to generate target sequence.

![]()<img src="https://cdn-images-1.medium.com/max/1400/1*Ismhi-muID5ooWf3ZIQFFg.png" width="700"/>

(image credit: [towardsdatascience.com](https://towardsdatascience.com))

To implement this model in DeepPavlov we have to code some DeepPavlov abstractions:
* **DatasetReader** to read the data
* **DatasetIterator** to generate batches
* **Vocabulary** to convert words to indexes
* **Model** to train it and then use it
* and some other components for pre- and postprocessing

### Probably the most usefull blog post about tensorflow I've seen
or why `tf.shape != tensor.get_shape`

[TensorFlow: Shapes and dynamic dimensions](https://blog.metaflow.fr/shapes-and-dynamic-dimensions-in-tensorflow-7b1fe79be363)

### Designations
    
for clarity we add the following suffixes to the end of python variables:

- `_ph` - tf.placeholder

- `_layer` - tf.keras.layer

- `_op` - tensorflow operation (remember that tf.Tensor is not a set of valuet, it is a node in computational graph).

In [0]:
import os
import pdb
import json
from itertools import chain
from pathlib import Path

import numpy as np
import tensorflow as tf

import deeppavlov
from deeppavlov import build_model
from deeppavlov.core.data.dataset_reader import DatasetReader
from deeppavlov.core.common.registry import register

In [3]:
tf.__version__, deeppavlov.__version__

('1.14.0', '0.4.0')

In [0]:
BATCH_SIZE = 32
MAXLEN = 50

## Download & extract dataset

In [5]:
from deeppavlov.core.data.utils import download_decompress

dataset_path = './personachat'

if not os.path.exists(dataset_path):
    download_decompress('http://files.deeppavlov.ai/datasets/personachat_v2.tar.gz', dataset_path)
else:
    print('Dataset has already been downloaded')

2019-07-14 03:02:54.356 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/personachat_v2.tar.gz to personachat/personachat_v2.tar.gz
I0714 03:02:54.356867 140328521566080 utils.py:63] Downloading from http://files.deeppavlov.ai/datasets/personachat_v2.tar.gz to personachat/personachat_v2.tar.gz
100%|██████████| 223M/223M [00:48<00:00, 4.59MB/s]
2019-07-14 03:03:43.16 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting personachat/personachat_v2.tar.gz archive into personachat
I0714 03:03:43.016792 140328521566080 utils.py:201] Extracting personachat/personachat_v2.tar.gz archive into personachat


## DatasetReader

DatasetReader is used to read and parse data from files. Here, we define new PersonaChatDatasetReader which reads [PersonaChat dataset](https://arxiv.org/abs/1801.07243).

PersonaChat dataset consists of dialogs and user personalities.

User personality is described by four sentences, e.g.:

    i like to remodel homes.
    i like to go hunting.
    i like to shoot a bow.
    my favorite holiday is halloween.
    
But we will be using only dialogues in this tutorial.

In [0]:
@register('personachat_dataset_reader')  # to use component later in train config it sould be registered
class PersonaChatDatasetReader(DatasetReader):
    """
    PersonaChat dataset from
    Zhang S. et al. Personalizing Dialogue Agents: I have a dog, do you have pets too?
    https://arxiv.org/abs/1801.07243
    Also, this dataset is used in ConvAI2 http://convai.io/
    This class reads dataset to the following format:
    [{
        'persona': [list of persona sentences],
        'x': input utterance,
        'y': output utterance,
        'dialog_history': list of previous utterances
        'candidates': [list of candidate utterances]
        'y_idx': index of y utt in candidates list
      },
       ...
    ]
    """
    def read(self, dir_path: str, mode='self_original', verbose=False):
        if verbose:
            print('Reading dataset...')
        dir_path = Path(dir_path)
        dataset = {}
        for dt in ['train', 'valid', 'test']:
            dataset[dt] = self._parse_data(dir_path / f'{dt}_{mode}.txt', verbose)

        print('Done\n')
        return dataset

    @staticmethod
    def _parse_data(filename, verbose):
        examples = []
        if verbose:
            print(filename)
        curr_persona = []
        curr_dialog_history = []
        persona_done = False
        with filename.open('r') as fin:
            for line in fin:
                line = ' '.join(line.strip().split(' ')[1:])
                your_persona_pref = 'your persona: '
                if line[:len(your_persona_pref)] == your_persona_pref and persona_done:
                    curr_persona = [line[len(your_persona_pref):]]
                    curr_dialog_history = []
                    persona_done = False
                elif line[:len(your_persona_pref)] == your_persona_pref:
                    curr_persona.append(line[len(your_persona_pref):])
                else:
                    persona_done = True
                    x, y, _, candidates = line.split('\t')
                    candidates = candidates.split('|')
                    example = {
                        'persona': curr_persona,
                        'x': x,
                        'y': y,
                        'dialog_history': curr_dialog_history[:],
                        'candidates': candidates,
                        'y_idx': candidates.index(y)
                    }
                    curr_dialog_history.extend([x, y])
                    examples.append(example)

        return examples

Read dataset, check size and sample some examples

In [7]:
data = PersonaChatDatasetReader().read('./personachat', verbose=True)

for k in data:
    print(k, '\t:', len(data[k]))

Reading dataset...
personachat/train_self_original.txt
personachat/valid_self_original.txt
personachat/test_self_original.txt
Done

train 	: 65719
valid 	: 7801
test 	: 7512


In [8]:
data['train'][100]

{'candidates': ['my girlfriend eloped w my best friend',
  'we should get a cleaner to clean out all the tofu my wife keeps cooking . gross !',
  'you sound young and fabulous . thanks for chatting !',
  'how do you like new york ?',
  'my sweet puppy will have the rings',
  'green ones ! ha ha . only because that is my favorite .',
  'i like to play soccer , i love games of throne and a vegan',
  'ah that is nice . like my turtles . you have pets ?',
  'peanut butter on my pizza',
  'which show were you in ?',
  'i still stay with my parents',
  'jokes sometimes goes serious and it is not good',
  'i play for midlands high school . our mascot is the cougars',
  'that sounds terrible ! but would probably make you rich .',
  'oh i like to all the time but everyone wants me everywhere',
  'i am super afraid of heights',
  'yeah , me and my grandma go there all the time .',
  'i like to attend social gatherings .',
  'i do not blame you . although i have to watch out for shellfish , i am 

### Dataset iterator

Dataset iterator is used to generate batches from parsed dataset (DatasetReader). Let's extract only `x` and `y` from parsed dataset and use them to predict sentence `y` by sentence `x`.

In [0]:
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator

@register('personachat_iterator')
class PersonaChatIterator(DataLearningIterator):
    def split(self, *args, **kwargs):
        for dt in ['train', 'valid', 'test']:
            setattr(self, dt, self._to_tuple(getattr(self, dt)))

    @staticmethod
    def _to_tuple(data):
        """
        Returns:
            list of (x, y)
        """
        return list(map(lambda x: (x['x'], x['y']), data))

In [10]:
iterator = PersonaChatIterator(data)
batch_generator = iterator.gen_batches(5, 'train')
batch = next(batch_generator)
for x, y in zip(*batch):
    print('x:', x)
    print('y:', y)
    print('----------')

x: how are you doing today ?
y: i am well , i am watching some anime and eating chocolate . how are you ?
----------
x: i have a bulldog what kind do you have
y: i have felix the cat and emmy the terrier my pride and joys at home
----------
x: aw , that sounds awful . let me guess , not compatible ? or did this person cheat ? !
y: he just does not like my glasses
----------
x: no . i wish i had a cat .
y: i take my dog for walks a lot .
----------
x: mine are fine but they still expect me to eat dinner every sunday with them .
y: boring . i am sorry for you , are you cook ?
----------


### Tokenizer
Splits utterance into tokens (words)

In [11]:
from deeppavlov.models.tokenizers.lazy_tokenizer import LazyTokenizer

tokenizer = LazyTokenizer()
tokenizer(["I'd like to tokenize some text"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


[['I', "'d", 'like', 'to', 'tokenize', 'some', 'text']]

### Vocabulary

Vocabulary object prepares mapping from tokens to token indices.
It uses train data to build this mapping.

We will implement DialogVocab (inherited from SimpleVocabulary) wich adds all tokens from `x` and `y` utterances to vocabulary.

In [0]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

@register('dialog_vocab')
class DialogVocab(SimpleVocabulary):
    def fit(self, *args):
        tokens = chain(*args)
        super().fit(tokens)

    def __call__(self, batch, **kwargs):
        indices_batch = []
        for utt in batch:
            tokens = [self[token] for token in utt]
            indices_batch.append(tokens)
        return indices_batch

Let's create instance of DialogVocab. We define save and load paths, minimal frequence of tokens which are added to vocabulary and set of special tokens.

Special tokens are:
* <PAD\> - padding
* <SOS\> - start of sequence
* <EOS\> - end of sequence
* <UNK\> - unknown token - token which is not presented in vocabulary

And fit it on tokens from *x* and *y*.

In [13]:
vocab = DialogVocab(
    save_path='./vocab.dict',
    load_path='./vocab.dict',
    min_freq=2,
    special_tokens=('<PAD>','<SOS>', '<EOS>', '<UNK>',),
    unk_token='<UNK>'
)

vocab.fit(tokenizer(iterator.get_instances(data_type='train')[0]), tokenizer(iterator.get_instances(data_type='train')[1]))
vocab.save()

PAD_idx = vocab._t2i['<PAD>']
SOS_idx = vocab._t2i['<SOS>']
assert PAD_idx == 0, 'this is required by tf.keras'

2019-07-14 03:04:16.500 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /content/vocab.dict]
I0714 03:04:16.500470 140328521566080 simple_vocab.py:89] [saving vocabulary to /content/vocab.dict]


In [14]:
# number of words in vocabhttps://github.com/text-machine-lab/ciss2_materials/blob/master/tutorials/deeppavlov_track/Tutorial_Day_2_seq2seq.ipynb
len(vocab)

11595

In [15]:
vocab.freqs.most_common(10)

[('i', 103487),
 ('.', 101599),
 ('you', 48296),
 ('?', 43771),
 (',', 39500),
 ('a', 34214),
 ('to', 32105),
 ('do', 30574),
 ('is', 28579),
 ('my', 26953)]

One can use vocabulary to encode tokenized text

In [16]:
vocab([['<SOS>', 'this', 'is', 'tokenized', 'there_is_no_such_word_in_dataset', 'and_this', 'sentence', '<EOS>', '<PAD>', '<PAD>']])

[[1, 110, 12, 3, 3, 3, 6060, 2, 0, 0]]

## Padding

To feed sequences of token indexes to neural model we should make their lengths equal. If sequence is too short we add <PAD\> symbols to the end of sequence. If sequence is too long we just cut it.

SentencePadder implements such behavior, it also adds <SOS\> and <EOS\> tokens.

In [0]:
from deeppavlov.core.models.component import Component

@register('sentence_padder')
class SentencePadder(Component):
    def __init__(self, length_limit, pad_token_id=0, start_token_id=1, end_token_id=2, *args, **kwargs):
        self.length_limit = length_limit
        self.pad_token_id = pad_token_id
        self.start_token_id = start_token_id
        self.end_token_id = end_token_id

    def __call__(self, batch):
        for i in range(len(batch)):
            batch[i] = batch[i][:self.length_limit]
            batch[i] = [self.start_token_id] + batch[i] + [self.end_token_id]
            batch[i] += [self.pad_token_id] * (self.length_limit + 2 - len(batch[i]))
        return batch

In [18]:
padder = SentencePadder(length_limit=6)
padded = padder(vocab(tokenizer(['this is very very long sentence that does not fit',
                                 'this one is short'])))
padded

[[1, 110, 12, 75, 75, 149, 6060, 2], [1, 110, 76, 12, 456, 2, 0, 0]]

To reverse mapping, just apply vocab again

In [19]:
vocab(padded)

[['<SOS>', 'this', 'is', 'very', 'very', 'long', 'sentence', '<EOS>'],
 ['<SOS>', 'this', 'one', 'is', 'short', '<EOS>', '<PAD>', '<PAD>']]

# Seq2seq model

![]()<img src="https://raw.githubusercontent.com/text-machine-lab/ciss2_materials/master/tutorials/deeppavlov_track/img/seq2seq_training.png" width="700"/>

(image credit: Stanford cs224n)

In [0]:
def build_seq2seq_graph(input_ph, target_ph, build_encoder_fn, build_decoder_fn, hidden_size, vocab_size, emb_size, dropout_rate, is_training_ph):
    """
    Args
        x_ph: input tokens placeholder
        y_ph: expected output tokens placeholder (used at training time for input feeding)
        build_encoder: function to build encoder graph
        build_decoder: function to build decoder graph
        hidden_dim: size of encoder rnn
        vocab_size: number of words in the vocabulary
        emb_dim: embedding size

    Returns:
        tf.Tensor [batch_size, maxlen, decoder_output_dim]
    """
    # embedding is shared between encoder and decoder
    embedding_layer = tf.keras.layers.Embedding(vocab_size, emb_size)
    dropout_layer = tf.keras.layers.Dropout(rate=dropout_rate)
    mask = tf.cast(input_ph, tf.bool)

    encoder_outputs_op, encoder_state_op = build_encoder_fn(
        input_ph,
        embedding_layer,
        dropout_layer,
        hidden_size,
        is_training_ph
    )
    decoder_op = build_decoder_fn(
        encoder_outputs_op,
        encoder_state_op,
        target_ph,
        embedding_layer,
        dropout_layer,
        is_training_ph,
        mask
    )
    logits_layer = tf.keras.layers.Dense(vocab_size)
    logits_op = logits_layer(decoder_op)
    return logits_op


In [0]:
def build_encoder(x_ph, embedding_layer, dropout_layer, hidden_dim, is_training_ph):
    """
    Args:
        x_ph: input tokens placeholder
        embedding_layer: tf.keras.Embedding object
        dropout_layer: tf.keras.Dropout object
        hidden_dim: size of rnn (also output size)
        is_training_ph: is training mode flag

    Returns:
        encoder_outputs_op: tf.Tensor, [batch_size, maxlen, encoder_hidden_size]
        encoder_state_op: tf.Tensor, [batch_size, encoder_hidden_size]
    """
    # embed x, apply dropout if is_training
    x_op = embedding_layer(x_ph)
    x_op = dropout_layer(x_op, training=is_training_ph)

    # make rnn layer and apply it to x
    # rnn layer should return both encoded sequences and state
    rnn_layer = tf.keras.layers.GRU(hidden_dim, return_sequences=True, return_state=True)
    encoder_outputs_op, encoder_state_op = rnn_layer(x_op)

    return encoder_outputs_op, encoder_state_op

Test encoder.

shapes should be `[batch_size, maxlen, encoder_hidden_size]` and `[batch_size, encoder_hidden_size]`

i.e `[3, 11, 17]` and `[3, 17]`

In [22]:
tf.reset_default_graph()

toy_batch_size = 3
toy_vocab_size = 13
toy_hidden_dim = 7
toy_emb_size = 5
toy_maxlen = 11

toy_emb = tf.keras.layers.Embedding(toy_vocab_size, toy_emb_size, input_length=toy_maxlen)
toy_dropout = tf.keras.layers.Dropout(rate=0.5)

# toy_input = tf.cast(tf.random_uniform(shape=[toy_batch_size, toy_maxlen]) * toy_vocab_size, tf.int32)
toy_input = tf.placeholder(tf.int64, [None, toy_maxlen])

encoder_outputs_op, encoder_state_op = build_encoder(
    toy_input, toy_emb, toy_dropout, toy_hidden_dim, True)

if (encoder_outputs_op.shape[1:] == (toy_maxlen, toy_hidden_dim)):
    print('Test 1 passed')
else:
    print('Problem with the shape')
    print(f'Shape should be {(None, toy_maxlen, toy_hidden_dim)}')
    print(f'But got {encoder_outputs_op.shape} instead')

if (encoder_state_op.shape[1:] == (toy_hidden_dim)):
    print('Test 2 passed')
else:
    print('Problem with the shape')
    print(f'Shape should be {(None, toy_hidden_dim)}')
    print(f'But got {encoder_state_op.shape} instead')

encoder_outputs_op, encoder_state_op

W0714 03:04:16.681995 140328521566080 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0714 03:04:16.716256 140328521566080 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Test 1 passed
Test 2 passed


(<tf.Tensor 'gru/transpose_1:0' shape=(?, 11, 7) dtype=float32>,
 <tf.Tensor 'gru/while/Exit_3:0' shape=(?, 7) dtype=float32>)

## Vanilla seq2seq decoder

In [0]:
def build_decoder(
        encoder_otputs_op,
        encoder_state_op,
        target_ph,
        embedding_layer,
        dropout_layer,
        is_training_ph,
        encoder_mask_ph=None):
    """Decoder without attention
    it ignores encoder_otputs_op and uses only encoder_state_op to generate sequence

    Args:
        encoder_otputs_op: used only to get max_len
        encoder_state_op: state of the encoder RNN, [batch_size, hidden_dim]
        enc_mask_ph: ignored
        target_ph: target placeholder, used at training time for input feeding
        embedding_layer: tf.keras.Embedding object
        dropout_layer: tf.keras.Dropout object
        is_training_ph: is training mode flag

    Returns:
        tf.Tensor [batch_size, max_len, hidden_dim]
    """
    _, max_len, hidden_dim = encoder_otputs_op.get_shape()
    _, max_len, hidden_dim = _, max_len.value, hidden_dim.value

    batch_size_op = tf.shape(encoder_otputs_op)[0]
    vocab_size = embedding_layer.input_dim

    # make decoder cell layer
    decoder_cell = tf.keras.layers.GRUCell(hidden_dim)
    # make decoder output projection layer (projects to vocabulary space)
    decoder_output_proj_layer = tf.keras.layers.Dense(vocab_size)

    # first decoder input is start-of-sentence token
    decoder_input_op = tf.ones([batch_size_op]) * SOS_idx
    # first decoder state is last encoder state
    decoder_state_op = encoder_state_op
    # in this list we will store the logits of predicted sequence
    output_logits = []

    for i in range(max_len):
        decoder_input_emb_op = embedding_layer(decoder_input_op)

        # for some complicated reasons, we must to expand_dims on state
        # decoder_cell returns output and states, in the case of GRU, states and output are the same
        decoder_state_op, _ = decoder_cell(decoder_input_emb_op, tf.expand_dims(decoder_state_op, 1))

        decoder_output_logit_op = decoder_output_proj_layer(decoder_state_op)
        output_logits.append(decoder_output_logit_op)

        # if training, use input feeding i.e. teacher forcing
        decoder_input_op = tf.cond(is_training_ph,
                                   lambda: target_ph[:, i],
                                   lambda: tf.argmax(decoder_output_logit_op, axis=1))

    output_logits_op = tf.stack(output_logits, axis=1)
    return output_logits_op

Test decoder

In [24]:
tf.reset_default_graph()

toy_batch_size = 3
toy_vocab_size = 13
toy_hidden_dim = 7
toy_emb_size = 5
toy_maxlen = 11

toy_emb = tf.keras.layers.Embedding(toy_vocab_size, toy_emb_size, input_length=toy_maxlen)
toy_dropout = tf.keras.layers.Dropout(rate=0.5)

toy_input = tf.placeholder(tf.int64, [None, toy_maxlen])
toy_target = tf.placeholder(tf.int64, [None, toy_maxlen])
toy_mask = tf.placeholder(tf.bool, [None, toy_maxlen])

is_training_ph = tf.placeholder_with_default(tf.constant(True), shape=())

encoder_outputs_op, encoder_state_op = build_encoder(
    toy_input, toy_emb, toy_dropout, toy_hidden_dim, is_training_ph)

toy_logits_op = build_decoder(
    encoder_outputs_op, encoder_state_op, toy_target, toy_emb, toy_dropout, is_training_ph, toy_mask
)

if (toy_logits_op.shape[1:] == (toy_maxlen, toy_vocab_size)):
    print('Test passed')
else:
    print('Problem with the shape')
    print(f'Shape should be {(None, toy_maxlen, toy_vocab_size)}')
    print(f'But got {toy_logits_op.shape} instead')

toy_logits_op

Test passed


<tf.Tensor 'stack:0' shape=(?, 11, 13) dtype=float32>

Test full model

In [0]:
toy_logits_op = build_seq2seq_graph(
    toy_input, toy_target, build_encoder, build_decoder, toy_hidden_dim, toy_vocab_size, toy_emb_size, 0.5, is_training_ph)

### Decoder with attention math

Decoder is much more tricky then encoder, especially with attention.
So it would be better for us to write down all decoder operations mathematically.

Let $m$ be the length of a source sequence, $h$ be dimension of encoder output, $\operatorname E \in \mathbb{R}^{ vocab\_size \times emb\_size}$ - embedding matrix.

Before encoding we have:
$$
\mathbf{h}_i^{enc} \in \mathbb{R}^h - \text{encoder output at i-th timestamp}\\
\mathbf{h}_m^{enc} - \text{last encoder output (encoder state)}\\
$$

#### Zeroth step

At the zeroth decoding step we sould construct decoder **input** and decoder **initial state**.
Decoder **initial state** is encoder state.
Decoder **input** is attention vector with SOS-token embedding as **query**.

Let $sos$ be SOS-token index in embedding matrix.

$$
\mathbf{h}_o^{dec} = \mathbf{h}_m\\
\mathbf{e}_0 = \operatorname{E}[sos]\\
% \mathbf{o}_0 = \mathbf 0, \mathbf{o}_0 \in \mathbb{R}^h\\
% \mathbf{h}_1^{dec} = \operatorname{Decoder}([e_0; o_0])
$$

#### t-th step, t > 0

At t-th step decoder **input** is attention vector with previous predicted token embedding as **query**.

**Note:** at training time we use **teacher forcing** (it is also called input feeding) that means that instead of using previous predicted token decoder uses previous true token from target sequence.


#### Attention

When we got decoder state $h_1$, we can compute attention vector.

Let $\operatorname{W}_{attProj} \in \mathbb{R}^{h \times h}$ be attention weighs, $\mathbf{s}$ be attention scores, $\mathbf{e}_t$ - embedding of the previous predicted token.

$$
\begin{aligned}
\mathbf{s}_{t, i} &= (\mathbf{e}_t)^T \operatorname{W}_{attProj} \mathbf{h}_i^{enc}\\
\mathbf{\alpha}_t &= \operatorname{Softmax}(\mathbf{s}_t) \text{  }\\
\mathbf{a}_t &= \sum_i^m \alpha_{t, i} \mathbf{h}_i^{enc}
\end{aligned}
$$

Or in terms of **query keys ans values**:

$$
\begin{aligned}
\mathbf{s}_{t, i} &= (\mathbf{q}_t)^T \operatorname{W}_{attProj} \mathbf{k}\\
\mathbf{\alpha}_t &= \operatorname{Softmax}(\mathbf{s}_t) \text{  }\\
\mathbf{a}_t &= \sum_i^m \alpha_{t, i} \mathbf{v}
\end{aligned}
$$

In [0]:
def softmax_masked(values, mask):
    masked_values = -np.inf * (1 - tf.cast(mask, tf.float32)) + values
    return tf.nn.softmax(masked_values, 2)

def build_decoder_with_attention(
        encoder_outputs_op,
        encoder_state_op,
        target_ph,
        embedding_layer,
        dropout_layer,
        is_training_ph,
        encoder_mask_ph):
    """Decoder with Luong attention
    https://arxiv.org/abs/1508.04025

    Args:
        encoder_otputs_op: used only to get max_len
        encoder_state_op: state of the encoder RNN, [batch_size, hidden_dim]
        enc_mask_ph: ignored
        target_ph: target placeholder, used at training time for input feeding
        embedding_layer: tf.keras.Embedding object
        dropout_layer: tf.keras.Dropout object
        is_training_ph: is training mode flag

    Returns:
        tf.Tensor [batch_size, max_len, hidden_dim]
    """    
    _, max_len, hidden_dim = encoder_outputs_op.get_shape()
    _, max_len, hidden_dim = _, max_len.value, hidden_dim.value

    batch_size_op = tf.shape(encoder_outputs_op)[0]
    vocab_size = embedding_layer.input_dim

    # make decoder cell layer
    decoder_cell = tf.keras.layers.GRUCell(hidden_dim)
    # make decoder output projection layer (projects to vocabulary space)
    decoder_output_proj_layer = tf.keras.layers.Dense(vocab_size)

    # first decoder input is start-of-sentence token
    decoder_input_op = tf.ones([batch_size_op], dtype=tf.int64) * SOS_idx
    # first decoder state is last encoder state
    decoder_state_op = encoder_state_op
    # in this list we will store the logits of predicted sequence
    output_logits = []

    # attention-related variables:
    attention_proj_layer = tf.keras.layers.Dense(hidden_dim, use_bias=False)
    attention_keys_op = attention_proj_layer(encoder_outputs_op)  # W_attProj @ h_enc
    attention_values_op = encoder_outputs_op
    attention_query_op = decoder_state_op

    for i in range(max_len):
        # compute input tensor for decoder rnn
        decoder_input_emb_op = embedding_layer(decoder_input_op)

        # apply attention with decoder_input_emb_op as query
        attention_query_op = tf.expand_dims(decoder_state_op, 1)  # [batch_size, 1, hidden]
        attention_scores_op = tf.matmul(attention_query_op, attention_keys_op, transpose_b=True)  # [batch_size, 1, maxlen]

        attention_probs_op = softmax_masked(attention_scores_op, tf.expand_dims(encoder_mask_ph, 1))
        attention_probs_op = tf.nn.softmax(attention_scores_op, 2)  # [batch_size, 1, maxlen]
        attention_vec_op = tf.matmul(attention_probs_op, attention_values_op)  # [batch_size, 1, hidden]
        attention_vec_op = tf.squeeze(attention_vec_op, 1)  # [batch_size, hidden]

        # for some complicated reasons, we must to expand_dims(decoder_state_op, 1) on state
        # decoder_cell returns output and states, in the case of GRU, states and output are the same
        decoder_state_op, _ = decoder_cell(attention_vec_op, tf.expand_dims(decoder_state_op, 1))

        decoder_output_logit_op = decoder_output_proj_layer(decoder_state_op)
        output_logits.append(decoder_output_logit_op)

        # if training, use input feeding i.e. teacher forcing
        decoder_input_op = tf.cond(is_training_ph,
                                   lambda: target_ph[:, i],
                                   lambda: tf.argmax(decoder_output_logit_op, axis=1))

    output_logits_op = tf.stack(output_logits, axis=1)
    return output_logits_op

In [57]:
tf.reset_default_graph()

toy_batch_size = 3
toy_vocab_size = 13
toy_hidden_dim = 7
toy_emb_size = 5
toy_maxlen = 11

toy_emb = tf.keras.layers.Embedding(toy_vocab_size, toy_emb_size)
toy_dropout = tf.keras.layers.Dropout(rate=0.5)

toy_input = tf.placeholder(tf.int64, [None, toy_maxlen])
toy_target = tf.placeholder(tf.int64, [None, toy_maxlen])
toy_mask = tf.placeholder(tf.bool, [None, toy_maxlen])

is_training_ph = tf.placeholder_with_default(tf.constant(True), shape=())

encoder_outputs_op, encoder_state_op = build_encoder(
    toy_input, toy_emb, toy_dropout, toy_hidden_dim, is_training_ph)

toy_logits_op = build_decoder_with_attention(
    encoder_outputs_op, encoder_state_op, toy_target, toy_emb, toy_dropout, is_training_ph, toy_mask
)

if (toy_logits_op.shape[1:] == (toy_maxlen, toy_vocab_size)):
    print('Test passed')
else:
    print('Problem with the shape')
    print(f'Shape should be {(toy_batch_size, toy_maxlen, toy_vocab_size)}')
    print(f'But got {toy_logits_op.shape} instead')

toy_logits_op

Test passed


<tf.Tensor 'stack:0' shape=(?, 11, 13) dtype=float32>

In [0]:
toy_logits_op = build_seq2seq_graph(
    toy_input, toy_target, build_encoder, build_decoder_with_attention, toy_hidden_dim, toy_vocab_size, toy_emb_size, 0.5, is_training_ph)

## Make model class and train

<img src="https://bastings.github.io/annotated_encoder_decoder/images/bahdanau.png" width="700"/>

In [0]:
from deeppavlov.core.models.tf_model import TFModel
# http://docs.deeppavlov.ai/en/master/_modules/deeppavlov/core/models/tf_model.html


@register('seq2seq_57389')
class Seq2Seq(TFModel):
    def __init__(self, **kwargs):
        # model hyperparameters
        self.emb_size = kwargs['emb_size']
        self.hidden = kwargs['hidden']
        self.dropout = kwargs['dropout']
        self.vocab_size = kwargs['vocab_size']
        self.max_len = kwargs['max_len']

        # optimization hyperparameters
        self.grad_clip = kwargs.get('grad_clip', 5.)
        self.learning_rate = kwargs.get('learning_rate', 1e-3)

        # placeholders
        self.input_ph = tf.placeholder(tf.int64, [None, self.max_len])
        self.target_ph = tf.placeholder(tf.int64, [None, self.max_len])
        self.is_training_ph = tf.placeholder_with_default(tf.constant(False), shape=())
        self.target_mask_ph = tf.cast(self.target_ph > 0, tf.float32)

        # graph
        self.logits_op = build_seq2seq_graph(
            self.input_ph,
            self.target_ph,
            build_encoder,
            build_decoder,#_with_attention,
            self.hidden,
            self.vocab_size,
            self.emb_size,
            self.dropout,
            self.is_training_ph)
        self.predictions_op = tf.argmax(self.logits_op, axis=2)

        self.loss = self._build_loss(self.input_ph, self.logits_op, self.target_mask_ph)

        self.train_op = self.get_train_op(self.loss,
                                          learning_rate=self.learning_rate,
                                          optimizer=tf.train.AdamOptimizer,
                                          clip_norm=self.grad_clip)

        # create session and initialize graph variables
        sess_config = tf.ConfigProto()
        sess_config.gpu_options.allow_growth = True  # do not use all GPU memory at once
        self.sess = tf.Session(config=sess_config)

        self.sess.run(tf.global_variables_initializer())

        super().__init__(**kwargs)
        if self.save_path:
            pass
        if self.load_path is not None:
            self.load()

    def _build_loss(self, y_true, y_logits_pred, y_mask):
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_logits_pred) * y_mask
        loss = tf.reduce_sum(loss) / tf.reduce_sum(y_mask)
        return loss

    def _build_feed_dict(self, x, y=None):
        feed_dict = {
            self.input_ph: x,
        }
        if y is not None:
            feed_dict.update({
                self.target_ph: y,
                self.is_training_ph: True,
            })
        return feed_dict

    def train_on_batch(self, x, y):
        feed_dict = self._build_feed_dict(x, y)
        loss, _ = self.sess.run([self.loss, self.train_op], feed_dict=feed_dict)
        return loss

    def __call__(self, x):
        feed_dict = self._build_feed_dict(x)
        y_pred = self.sess.run(self.predictions_op, feed_dict=feed_dict)
        return y_pred

    def process_event(self, *args, **kwargs):
        pass

In [60]:
model = Seq2Seq(emb_size=3, hidden=5, dropout=0.1, vocab_size=7, max_len=9, save_path=None)

2019-07-14 04:15:06.803 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 50: No load path is set for Seq2Seq!
W0714 04:15:06.803834 140328521566080 serializable.py:50] No load path is set for Seq2Seq!


### Postprocessing

In postprocessing step we are going to remove all <PAD\>, <SOS\>, <EOS\> tokens.

In [0]:
@register('postprocessing')
class SentencePostprocessor(Component):
    def __init__(self, pad_token='<PAD>', start_token='<SOS>', end_token='<EOS>', *args, **kwargs):
        self.pad_token = pad_token
        self.start_token = start_token
        self.end_token = end_token

    def __call__(self, batch):
        for i in range(len(batch)):
            batch[i] = ' '.join(self._postproc(batch[i]))
        return batch
    
    def _postproc(self, utt):
        if self.end_token in utt:
            utt = utt[:utt.index(self.end_token)]
        return utt

In [0]:
postprocess = SentencePostprocessor()

In [0]:
padder(vocab([['hello', 'my', 'friend', 'there_is_no_such_word_in_dataset', 'and_this'], ['It']]))

In [0]:
padder = SentencePadder(length_limit=9 - 2)
model = Seq2Seq(emb_size=3, hidden=5, dropout=0.1, vocab_size=7, max_len=9, save_path=None)
postprocess(vocab(model(padder(vocab([['hello', 'my', 'friend', 'there_is_no_such_word_in_dataset', 'and_this']])))))

### Create config file
Let's put is all together in one config file.

In [0]:
config = {
  "dataset_reader": {
    "class_name": "personachat_dataset_reader",
    "data_path": "./personachat"
  },
  "dataset_iterator": {
    "class_name": "personachat_iterator",
    "shuffle": True
  },
  "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "class_name": "lazy_tokenizer",
        "id": "tokenizer",
        "in": ["x"],
        "out": ["x_tokens"]
      },
      {
        "class_name": "lazy_tokenizer",
        "id": "tokenizer",
        "in": ["y"],
        "out": ["y_tokens"]
      },
      {
        "class_name": "dialog_vocab",
        "id": "vocab",
        "save_path": "./vocab.dict",
        "load_path": "./vocab.dict",
        "min_freq": 2,
        "special_tokens": ["<PAD>","<SOS>", "<EOS>", "<UNK>"],
        "unk_token": "<UNK>",
        "fit_on": ["x_tokens", "y_tokens"],
        "in": ["x_tokens"],
        "out": ["x_tokens_ids"]
      },
      {
        "ref": "vocab",
        "in": ["y_tokens"],
        "out": ["y_tokens_ids"]
      },
      {
        "class_name": "sentence_padder",
        "id": "padder",
        "length_limit": MAXLEN,
        "in": ["x_tokens_ids"],
        "out": ["x_tokens_ids"]
      },
      {
        "ref": "padder",
        "in": ["y_tokens_ids"],
        "out": ["y_tokens_ids"]
      },
      {
        "class_name": "seq2seq_57389",
        "id": "seq2seq_model",
        "max_len": "#padder.length_limit+2",
        "hidden": 250,
        "emb_size": 100,
        "vocab_size": len(vocab),
        "dropout": 0.1,
        "learning_rate": 1e-3,
        "save_path": "./seq2seq_model_57389",
        "load_path": "./seq2seq_model_57389",
        "in": ["x_tokens_ids"],
        "in_y": ["y_tokens_ids"],
        "out": ["y_predicted_tokens_ids"],
      },
      {
        "ref": "vocab",
        "in": ["y_predicted_tokens_ids"],
        "out": ["y_predicted_tokens"]
      },
      {
        "class_name": "postprocessing",
        "in": ["y_predicted_tokens"],
        "out": ["y_predicted_tokens"]
      }
    ],
    "out": ["y_predicted_tokens"]
  },
  "train": {
    "log_every_n_batches": 100,
    "val_every_n_epochs": 500,
    "batch_size": 64,
    "validation_patience": 5,
    "epochs": 10,
    "max_batches": 1000,
    "metrics": ["bleu"],
  }
}

### Interact with model using config

In [0]:
model = build_model(config)

In [39]:
model(['hi, how are you?', 'any ideas my dear friend?'])

['painters genre tina tina tina tina learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn',
 'painters tina tina tina tina tina learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn learn']

### Train model


Run experiments with and without attention, with teacher forcing and without.

In [0]:
from deeppavlov import train_model

train_model(config)

2019-07-14 04:15:41.44 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /content/vocab.dict]
I0714 04:15:41.044623 140328521566080 simple_vocab.py:103] [loading vocabulary from /content/vocab.dict]


Done



2019-07-14 04:16:05.316 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 89: [saving vocabulary to /content/vocab.dict]
I0714 04:16:05.316766 140328521566080 simple_vocab.py:89] [saving vocabulary to /content/vocab.dict]


In [45]:
model = build_model(config)

model(['hi, how are you?', 'any ideas my dear friend?', 'okay, i agree with you', 'good bye!'])

2019-07-14 03:35:44.876 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /content/vocab.dict]
I0714 03:35:44.876637 140328521566080 simple_vocab.py:103] [loading vocabulary from /content/vocab.dict]
2019-07-14 03:36:05.529 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /content/seq2seq_model_57389]
I0714 03:36:05.529874 140328521566080 tf_model.py:52] [loading model from /content/seq2seq_model_57389]


['frend shore shore approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching',
 'corporation shore shore approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching appr

In [42]:
model(['tell me about yourself'])

['corporation shore shore approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching']

## Telegram bot

In [0]:
# to interact with CLI
from deeppavlov.core.commands.infer import interact_model
# to interact with Telegram
from deeppavlov.utils.telegram.telegram_ui import interact_model_by_telegram

In [44]:
interact_model(config)

2019-07-14 03:31:51.928 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /content/vocab.dict]
I0714 03:31:51.928510 140328521566080 simple_vocab.py:103] [loading vocabulary from /content/vocab.dict]
2019-07-14 03:32:12.992 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /content/seq2seq_model_57389]
I0714 03:32:12.992400 140328521566080 tf_model.py:52] [loading model from /content/seq2seq_model_57389]


x::hi
>> corporation shore shore approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching approaching
x::exit


In [0]:
interact_model_by_telegram(config,
                           token='YOUR_TOKEN')

## Extra
### Decoder with attention math 2 (more realistic case)

This is typical NMT decoder with attention. It uses a lot of hacky tricks to make decoding a bit better.

Decoder is much more tricky then encoder, especially with attention.
So it would be better for us to write down all decoder operations mathematically.

Let $m$ be the length of a source sequence, $h$ be dimension of encoder output, $\operatorname E \in \mathbb{R}^{ vocab\_size \times emb\_size}$ - embedding matrix.

Before encoding we have:
$$
\mathbf{h}_i^{enc} \in \mathbb{R}^h - \text{encoder output at i-th timestamp}\\
\mathbf{h}_m^{enc} - \text{last encoder output (encoder state)}\\
$$

#### Zeroth step

At the zeroth decoding step we sould construct decoder **input** and decoder **initial state**.
Decoder **initial state** is transformed (projected with matrix $\mathbf{\operatorname{W}}_h$) encoder state.
Decoder **input** is _zero_ vector of size $h$.

Let $sos$ be SOS-token index in embedding matrix.

$$
\mathbf{h}_o^{dec} = \operatorname{W}_h \mathbf{h}_m, \; \operatorname{W}_h \in \mathbb{R}^{h \times 2h} - \text{decoder initial state is transformed encoder state}\\
\mathbf{e}_0 = \operatorname{E}[sos]\\
\mathbf{o}_0 = \mathbf 0, \mathbf{o}_0 \in \mathbb{R}^h\\
\mathbf{h}_1^{dec} = \operatorname{Decoder}([e_0; o_0])
$$

#### t-th step, t > 0

At t-th step decoder **input** is concatenated combined-output vector $\mathbf{o}_t$ (it is explained down this page in Attention paragraph) and previous predicted token embedding.

**Note:** at training time we use **teacher forcing** (it is also called input feeding) that means that instead of using previous predicted token decoder uses previous true token from target sequence.


#### Attention

When we got decoder state $h_1$, we can compute attention vector.

Let $\operatorname{W}_{attProj} \in \mathbb{R}^{h \times 2h}$ be attention weighs, $\mathbf{s}$ be attention scores.

$$
\begin{aligned}
\mathbf{s}_{t, i} &= (h_t^{dec})^T \operatorname{W}_{attProj} h_i^{enc}\\
\mathbf{\alpha}_t &= \operatorname{Softmax}(\mathbf{s}_t) \text{  }\\
\mathbf{a}_t &= \sum_i^m \alpha_{t, i} \mathbf{h}_i^{enc}
\end{aligned}
$$

Then, decoder output is concatenated with attention vector and passed through a linear layer, tanh and dropout to attain combined-output vector $\mathbf{o}_t$.

$$
\begin{aligned}
\mathbf{u}_t = [a_t; h_t^{dec}]
    \text{   } \; &where \; \text{  }
        \mathbf{u}_t \in \mathbb{R}^{3h \times 1}\\
\mathbf{o}_t = \operatorname{Dropout(tanh(W_u} \mathbf{u}_t))
    \text{   } \;  &where  \text{   } \; 
        \operatorname{W}_u \in \mathbb{R}^{h \times 3h}, \mathbf{o}_t \in \mathbb{R}^{h \times 1}
\end{aligned}
$$